In [0]:
!pip install git+https://www.github.com/bojone/bert4keras.git
!pip install keras-lr-finder
!pip install numpy==1.16.2
!pip install git+https://github.com/TinkerMob/keras_albert_model.git
!pip install keras-bert

In [11]:
from bert4keras.backend import keras
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model

from keras_albert_model import build_albert, load_brightmart_albert_zh_checkpoint

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import cv2
from os import listdir
from sklearn.metrics import accuracy_score
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras import backend as K

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) #command to run codeon multiple gpu

from keras_lr_finder import LRFinder
from pprint import pprint
tf.logging.set_verbosity(tf.logging.ERROR)
from random import choice
import gc
from keras_bert.layers import Extract
from keras_bert import get_pretrained, PretrainedList, get_checkpoint_paths, load_trained_model_from_checkpoint
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/data")

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
num_classes = 2
maxlen = 196
batch_size = 32
config_path = 'albert_small_zh_google/albert_config_small_google.json'
checkpoint_path = 'albert_small_zh_google/albert_model.ckpt'
dict_path = 'albert_small_zh_google/vocab.txt'

In [0]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [13]:
model = load_brightmart_albert_zh_checkpoint('albert_xlarge_zh')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embed-Token (AdaptiveEmbedding) [(None, 512, 2048),  2966528     Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Segment (Embedding)       (None, 512, 2048)    4096        Input-Segment[0][0]              
____________________________________________________________________________________________

In [98]:
albert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=True,
)
# tokens = keras.layers.Input(shape=(maxlen,))
# segments = keras.layers.Input(shape=(maxlen,))
# albert_input = [tokens, segments]
# text_output = albert(albert_input)

# albert = load_brightmart_albert_zh_checkpoint('albert_xlarge_zh')
# albert = build_albert(token_num=30000, training=False, output_layers=[-1, -2, -3, -4])
text_output = albert.output

extract = Extract(index=-1, name='Extract')(text_output)
text_repr = keras.layers.Dense(32, activation='relu')(extract)

conv_base = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)) #include_top 是否包含全连接层
conv_base.trainable=False
images = keras.layers.Input(shape=(224,224,3))
base_output = conv_base(images)
flat = keras.layers.Flatten()(base_output)

visual_repr = keras.layers.Dense(32,activation='relu')(flat)

#classifier
combine_repr = keras.layers.concatenate([text_repr, visual_repr])
com_drop=keras.layers.Dropout(0.3)(combine_repr)

com_drop = keras.layers.Dense(35, activation='relu')(com_drop)
com_drop = keras.layers.Dropout(0.3)(com_drop)

output = keras.layers.Dense(1, activation='sigmoid')(com_drop)

model = keras.models.Model(inputs=[albert.input[0], albert.input[1], images], outputs=output)

model.summary()

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 2048)   43270144    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 2048)   4096        Input-Segment[0][0]              
___________________________________________________________________________________________

In [0]:
data_image = np.load("data_image.npy")
data_text = np.load("data_text.npy")
data_label = np.load("data_label.npy")


train_text = data_text[:15360]
trainY = data_label[:15360]
test_text = data_text[15362:]
testY = data_label[15362:]

train_imagesX = np.load('train_imagesX.npy')
test_imagesX = np.load('test_imagesX.npy')

len(train_text), len(test_text), len(trainY), len(testY), len(train_imagesX)

(15360, 3840, 15360, 3840, 15361)

In [0]:
def generate_sequence(text, images, Y):
    tokens, segments = [], []
    
    for i in text:
      token_ids, segment_ids = tokenizer.encode(i, max_length=maxlen)
      tokens.append(token_ids)
      segments.append(segment_ids)

    return (tokens, segments, images, Y)


train_tokens, train_segments, train_images, trainy = generate_sequence(train_text, train_imagesX, trainY)

test_tokens, test_segments, test_images, testy = generate_sequence(test_text, test_imagesX, testY)

In [0]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [0]:
train_images = train_images[:15360]
test_images = test_images[1:]

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00005),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

### 定义callback函数，只保留val_sparse_categorical_accuracy 得分最高的模型

from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("MMFFND-albert-Finetune.h5", monitor='accuracy', verbose=1, save_best_only=True,
                            mode='max')

model.fit(
    x=[train_tokens, train_segments, train_images], 
    y=trainy,
    batch_size=batch_size,
    epochs=5,
    verbose=1,
    shuffle=False,
    validation_split=0.3,
    callbacks=[checkpoint]
)

ValueError: ignored